In [1]:
import pymysql
from xlrd import open_workbook
import os
import re

In [ ]:
connection = pymysql.connect(host='localhost',
                                 user='cat',
                                 password='12345678',
                                 db='cat',
                                 charset='utf8',
                                 cursorclass=pymysql.cursors.DictCursor)

In [ ]:
cs = connection.cursor()


In [ ]:
sql = ("SELECT * FROM cat.collocations_2_grams_economics LIMIT 1000;")
cs.execute(sql)
result = cs.fetchall()
print(result)

In [ ]:
files = os.listdir('C:/Users/E6230/Downloads/Частотные списки-20190328T093323Z-001/Частотные списки/coll')
# for domain in files:
#     domain = domain.split('_')[0]
#     for i in range(2, 7):
#         tablename = "collocations_{0}_grams_{1}".format(i, domain)
#         cs.execute("SHOW TABLES")
#         result = cs.fetchall()
#         tables = [list(dictionary.values())[0] for dictionary in result]
#         if tablename not in tables:
#             create_table = "CREATE TABLE IF NOT EXISTS " + tablename + " (id_collocation INT AUTO_INCREMENT, occurence VARCHAR(255), lemma VARCHAR(255), POS_tag VARCHAR(255), absolute_frequency FLOAT, normalized_frequency FLOAT, PMI FLOAT, log_likelihood FLOAT, t_score FLOAT, PRIMARY KEY (id_collocation))"
#             cs.execute(create_table)
for file in files:
    print(file)
    data = open_workbook(filename =
                         os.path.join('C:/Users/E6230/Downloads/Частотные списки-20190328T093323Z-001/Частотные списки/coll',
                                      file))
    domain = file.split('_')[0]
    for i, sheetname in enumerate(('bigrams', 'trigrams', 'quadrograms', 'fivegrams', 'sixgrams')):
        ngrams = data.sheet_by_name(sheetname)
        for r in range(1, ngrams.nrows):
            occurence = ngrams.cell(r, 3).value
            pos = ngrams.cell(r, 7).value
            abs_freq = ngrams.cell(r, 1).value
            norm_freq = ngrams.cell(r, 5).value
            pmi = ngrams.cell(r, 4).value
            log_like = ngrams.cell(r, 2).value
            t_score = ngrams.cell(r, 6).value
            values = (occurence, pos, abs_freq, norm_freq, pmi, log_like, t_score)
            cs.execute("INSERT INTO " + "collocations_{0}_grams_{1}".format(
                i + 2,
                domain) + " (occurence, POS_tag, absolute_frequency, normalized_frequency, PMI, log_likelihood, t_score) VALUES (%s, %s, %s, %s, %s, %s, %s)",
                        values)
connection.commit()

In [ ]:
create_table_metas = "CREATE TABLE IF NOT EXISTS meta_cat (id_text INT AUTO_INCREMENT, domain VARCHAR(255), year INT, author VARCHAR(255), journal VARCHAR(255), source VARCHAR(255), article_name VARCHAR(255), PRIMARY KEY (id_text))"
cs.execute(create_table_metas)
create_table_words = "CREATE TABLE IF NOT EXISTS words_cat (id_word INT AUTO_INCREMENT, abs_sent_id INT, id_sent INT, id_text INT, word VARCHAR(255), lemma VARCHAR(255), POS_tag VARCHAR(255), morph VARCHAR(255), root_dist INT, sint_role VARCHAR(255), PRIMARY KEY (id_word), FOREIGN KEY foreign_key_id_text(id_text) REFERENCES meta_cat(id_text) ON UPDATE CASCADE ON DELETE CASCADE)"
cs.execute(create_table_words)
connection.commit()

In [ ]:
conll_dir = 'C:\\Users\\E6230\\udpiped\\linguistics'
metadir = 'C:\\Users\\E6230\\old_meta\\metas\\meta_dialogue2017'

In [ ]:
for meta in os.listdir(metadir):
    #print(meta)
    with open(os.path.join(metadir,meta),'r',encoding='utf-8') as text:
        lines = text.readlines()
        metadata = []
        for line in lines:
            if line != '' and line != '\n':
                metadata.append(line)
        cs.execute(
            "INSERT INTO meta_cat (domain, year, author, journal, source, article_name) VALUES (%s, %s, %s, %s, %s, %s)",
            (metadata[4], metadata[3], metadata[2], metadata[5], metadata[0], metadata[1]))
    get_last_metas = "SELECT MAX(id_text) FROM meta_cat"
    cs.execute(get_last_metas)
    idtext = cs.fetchall()[0]['MAX(id_text)']
    conll = '2017_{}.txt'.format(str(int(meta.split('-')[-1].replace('.txt',''))-1))
    #print(conll)
    with open(os.path.join(conll_dir, conll),'r',encoding='utf-8') as text:
        lines = text.readlines()
        abs_sent_id = 1  # к какому по счету предложению в тексте относится слово
        for line in lines:
            if line != '' and line != '\n':
                if line[0] != '#':  # в формате udpipe номер предложения и текст закоменчены #
                    spl_line = line.split('\t')
                    cs.execute(
                        "INSERT INTO words_cat (id_text, abs_sent_id, id_sent, word, lemma, POS_tag, morph, root_dist, sint_role) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                        (idtext, abs_sent_id, spl_line[0], spl_line[1], spl_line[2], spl_line[3], spl_line[5],
                         spl_line[6], spl_line[7]))
            else:
                abs_sent_id += 1
                
connection.commit()

In [17]:
conll_dir = 'C:\\Users\\E6230\\udpiped\\hse_law_2017_cleaned'
metadir = 'C:\\Users\\E6230\\old_meta\\articles\\hse_law_2017_articles_meta'
connection = pymysql.connect(host='localhost',
                                 user='cat',
                                 password='12345678',
                                 db='cat',
                                 charset='utf8',
                                 cursorclass=pymysql.cursors.DictCursor)
cs = connection.cursor()
for meta in os.listdir(metadir):
    print(meta)
    with open(os.path.join(metadir,meta),'r',encoding='utf-8') as text:
        lines = text.read().splitlines()
        print(lines)
        if len(lines)==6:
            metadata = [lines[0]]
            metadata.append(lines[1]+' '+lines[2].replace('\n',' '))
            for line in lines[3:]:
                if line != '':
                    metadata.append(line)
        elif len(lines)>6:
            metadata = [lines[0]]
            metadata.append(lines[1]+' '+lines[2]+' '+lines[3].replace('\n',' '))
            for line in lines[4:]:
                if line != '' and line != '\n':
                    metadata.append(line)
        else:
            metadata=[line for line in lines if line != '' and line != '\n']
        print(metadata)
        cs.execute(
            "INSERT INTO meta_cat (domain, year, author, journal, source, article_name) VALUES (%s, %s, %s, %s, %s, %s)",
            ('Право', 2017, metadata[2], 'Право. Журнал Высшей школы экономики', metadata[0], metadata[1]))
    get_last_metas = "SELECT MAX(id_text) FROM meta_cat"
    cs.execute(get_last_metas)
    idtext = cs.fetchall()[0]['MAX(id_text)']
    conll = meta.replace('_meta','')
    print(conll)
    with open(os.path.join(conll_dir, conll),'r',encoding='utf-8') as text:
        lines = text.read().splitlines()
        abs_sent_id = 1  # к какому по счету предложению в тексте относится слово
        for line in lines:
            if line != '' and line != '\n':
                if line[0] != '#':  # в формате udpipe номер предложения и текст закоменчены #
                    spl_line = line.split('\t')
                    cs.execute(
                        "INSERT INTO words_cat (id_text, abs_sent_id, id_sent, word, lemma, POS_tag, morph, root_dist, sint_role) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                        (idtext, abs_sent_id, spl_line[0], spl_line[1], spl_line[2], spl_line[3], spl_line[5],
                         spl_line[6], spl_line[7]))
            else:
                abs_sent_id += 1
                
connection.commit()

hse_2017_10_meta.txt
['https://law-journal.hse.ru/data/2017/12/29/1160720926/%D1%81%D0%B0%D0%B7%D0%BE%D0%BD%D0%BE%D0%B2%D0%B0.pdf', '«Гибридная война»:международно-правовое измерение', 'Сазонова К. Л.', '2017', 'Право']
['https://law-journal.hse.ru/data/2017/12/29/1160720926/%D1%81%D0%B0%D0%B7%D0%BE%D0%BD%D0%BE%D0%B2%D0%B0.pdf', '«Гибридная война»:международно-правовое измерение', 'Сазонова К. Л.', '2017', 'Право']
hse_2017_10.txt
hse_2017_11_meta.txt
['https://law-journal.hse.ru/data/2017/12/29/1160721340/%D1%87%D1%83%D0%BA%D1%80%D0%B5%D0%B5%D0%B2.pdf', 'Доктрина фрустрации в английском договорном праве', 'Чукреев А.', '2017', 'Право']
['https://law-journal.hse.ru/data/2017/12/29/1160721340/%D1%87%D1%83%D0%BA%D1%80%D0%B5%D0%B5%D0%B2.pdf', 'Доктрина фрустрации в английском договорном праве', 'Чукреев А.', '2017', 'Право']
hse_2017_11.txt
hse_2017_1_meta.txt
['https://law-journal.hse.ru/data/2017/12/29/1160719067/%D1%82%D0%B8%D1%85%D0%BE%D0%BC%D0%B8%D1%80%D0%BE%D0%B2.pdf', 'Правовое про

In [13]:
conll_dir = 'C:\\Users\\E6230\\udpiped\\spbu_politology_2016_cleaned'
metadir = 'C:\\Users\\E6230\\old_meta\\articles\\spbu_politology_articles_2016_meta'
connection = pymysql.connect(host='localhost',
                                 user='cat',
                                 password='12345678',
                                 db='cat',
                                 charset='utf8',
                                 cursorclass=pymysql.cursors.DictCursor)
cs = connection.cursor()
for meta in os.listdir(metadir):
    print(meta)
    with open(os.path.join(metadir,meta),'r',encoding='utf8') as text:
        lines = text.read().splitlines()
        print(lines)
        if len(lines)==6:
            metadata = [lines[0]]
            metadata.append(lines[1]+' '+lines[2].replace('\n',' '))
            for line in lines[3:]:
                if line != '':
                    metadata.append(line)
        elif len(lines)>6:
            metadata = [lines[0]]
            metadata.append(lines[1]+' '+lines[2]+' '+lines[3].replace('\n',' '))
            for line in lines[4:]:
                if line != '' and line != '\n':
                    metadata.append(line)
        else:
            metadata=[line for line in lines if line != '' and line != '\n']
        print(metadata)
        cs.execute(
            "INSERT INTO meta_cat (domain, year, author, journal, source, article_name) VALUES (%s, %s, %s, %s, %s, %s)",
            ('Политология', 2016, metadata[2], 'Вестник Санкт-Петербургского университета. ПОЛИТОЛОГИЯ. МЕЖДУНАРОДНЫЕ ОТНОШЕНИЯ.', metadata[0], metadata[1]))
    get_last_metas = "SELECT MAX(id_text) FROM meta_cat"
    cs.execute(get_last_metas)
    idtext = cs.fetchall()[0]['MAX(id_text)']
    conll = meta.replace('_meta','')
    print(conll)
    with open(os.path.join(conll_dir, conll),'r',encoding='utf-8') as text:
        lines = text.read().splitlines()
        abs_sent_id = 1  # к какому по счету предложению в тексте относится слово
        for line in lines:
            if line != '' and line != '\n':
                if line[0] != '#':  # в формате udpipe номер предложения и текст закоменчены #
                    spl_line = line.split('\t')
                    cs.execute(
                        "INSERT INTO words_cat (id_text, abs_sent_id, id_sent, word, lemma, POS_tag, morph, root_dist, sint_role) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                        (idtext, abs_sent_id, spl_line[0], spl_line[1], spl_line[2], spl_line[3], spl_line[5],
                         spl_line[6], spl_line[7]))
            else:
                abs_sent_id += 1
                
connection.commit()

spbu_politology_2016_v1_11_meta.txt
['http://vestnik.spbu.ru/html16/s06/s06v1/03.pdf', 'Политическая конкуренция vs политическая стабильность ', '\tв современной России', 'Макарин А. В.', '2016', 'Политология и международные отношения']
['http://vestnik.spbu.ru/html16/s06/s06v1/03.pdf', 'Политическая конкуренция vs политическая стабильность  \tв современной России', 'Макарин А. В.', '2016', 'Политология и международные отношения']
spbu_politology_2016_v1_11.txt
spbu_politology_2016_v1_14_meta.txt
['http://vestnik.spbu.ru/html16/s06/s06v1/04.pdf', 'Сетевые подходы к управлению в сфере контроля над преступностью', 'Гуринская А. Л.', '2016', 'Политология и международные отношения']
['http://vestnik.spbu.ru/html16/s06/s06v1/04.pdf', 'Сетевые подходы к управлению в сфере контроля над преступностью', 'Гуринская А. Л.', '2016', 'Политология и международные отношения']
spbu_politology_2016_v1_14.txt
spbu_politology_2016_v1_17_meta.txt
['http://vestnik.spbu.ru/html16/s06/s06v1/05.pdf', 'К вопро

spbu_politology_2016_v2_27_meta.txt
['http://vestnik.spbu.ru/html16/s06/s06v2/08.pdf', 'Некоторые аспекты международного экономического', 'сотрудничества в Арктике', 'Пилясов А. Н., Котов А. В.', '2016', 'Политология и международные отношения']
['http://vestnik.spbu.ru/html16/s06/s06v2/08.pdf', 'Некоторые аспекты международного экономического сотрудничества в Арктике', 'Пилясов А. Н., Котов А. В.', '2016', 'Политология и международные отношения']
spbu_politology_2016_v2_27.txt
spbu_politology_2016_v2_30_meta.txt
['http://vestnik.spbu.ru/html16/s06/s06v2/09.pdf', 'Методология исследования основ международного ', '\tэкономического сотрудничества в Арктике', 'Харлампьева Н. К.', '2016', 'Политология и международные отношения']
['http://vestnik.spbu.ru/html16/s06/s06v2/09.pdf', 'Методология исследования основ международного  \tэкономического сотрудничества в Арктике', 'Харлампьева Н. К.', '2016', 'Политология и международные отношения']
spbu_politology_2016_v2_30.txt
spbu_politology_2016_v

spbu_politology_2016_v4_17_meta.txt
['http://vestnik.spbu.ru/html16/s06/s06v4/05.pdf', 'Университеты в политике: ', '\tк вопросу о перспективах исследования в области политической теории', 'Филимонов К. Г.', '2016', 'Политология и международные отношения']
['http://vestnik.spbu.ru/html16/s06/s06v4/05.pdf', 'Университеты в политике:  \tк вопросу о перспективах исследования в области политической теории', 'Филимонов К. Г.', '2016', 'Политология и международные отношения']
spbu_politology_2016_v4_17.txt
spbu_politology_2016_v4_24_meta.txt
['http://vestnik.spbu.ru/html16/s06/s06v4/07.pdf', 'Образование в области международных отношений в России', 'Лебедева М. М.', '2016', 'Политология и международные отношения']
['http://vestnik.spbu.ru/html16/s06/s06v4/07.pdf', 'Образование в области международных отношений в России', 'Лебедева М. М.', '2016', 'Политология и международные отношения']
spbu_politology_2016_v4_24.txt
spbu_politology_2016_v4_27_meta.txt
['http://vestnik.spbu.ru/html16/s06/s06

In [ ]:
conll_dir = 'C:\\Users\\E6230\\udpiped\\economics'
metadir = 'C:\\Users\\E6230\\old_meta\\meta_economics/meta'
connection = pymysql.connect(host='localhost',
                                 user='cat',
                                 password='12345678',
                                 db='cat',
                                 charset='utf8',
                                 cursorclass=pymysql.cursors.DictCursor)
cs = connection.cursor()
passed = set()
for meta in os.listdir(metadir):   
    conll = os.path.join(conll_dir, meta)
    if os.path.exists(conll):
        print(meta)
        with open(os.path.join(metadir,meta),'r',encoding='utf-8') as text:
            metadata = text.read().replace('\ufeff','').splitlines()
            
            article_name = metadata[1]
            if article_name in passed:
                continue
            print(metadata)
            passed.add(article_name)
            cs.execute(
                "INSERT INTO meta_cat (domain, year, author, journal, source, article_name) VALUES (%s, %s, %s, %s, %s, %s)",
                (metadata[4], int(metadata[3]), metadata[2].strip(), metadata[5], metadata[0], article_name))
        get_last_metas = "SELECT MAX(id_text) FROM meta_cat"
        cs.execute(get_last_metas)
        idtext = cs.fetchall()[0]['MAX(id_text)']

        with open(conll,'r',encoding='utf-8') as text:
            lines = text.read().splitlines()
            abs_sent_id = 1  # к какому по счету предложению в тексте относится слово
            for line in lines:
                if line != '' and line != '\n':
                    if line[0] != '#':  # в формате udpipe номер предложения и текст закоменчены #
                        spl_line = line.split('\t')
                        cs.execute(
                            "INSERT INTO words_cat (id_text, abs_sent_id, id_sent, word, lemma, POS_tag, morph, root_dist, sint_role) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                            (idtext, abs_sent_id, spl_line[0], spl_line[1], spl_line[2], spl_line[3], spl_line[5],
                             spl_line[6], spl_line[7]))
                else:
                    abs_sent_id += 1
                
connection.commit()

In [ ]:
conll_dir = 'C:\\Users\\E6230\\udpiped\\spbu_sociology_2017'
metadir = 'C:\\Users\\E6230\\old_meta\\metas\\spbu_sociology_2017_meta'
connection = pymysql.connect(host='localhost',
                                 user='cat',
                                 password='12345678',
                                 db='cat',
                                 charset='utf8',
                                 cursorclass=pymysql.cursors.DictCursor)
cs = connection.cursor()
for meta in os.listdir(metadir):
    print(meta)
    with open(os.path.join(metadir,meta),'r',encoding='utf-8') as text:
        lines = text.read().splitlines()
        print(lines)
        if len(lines)==6:
            metadata = [lines[0]]
            metadata.append(lines[1]+' '+lines[2].replace('\n',' '))
            for line in lines[3:]:
                if line != '':
                    metadata.append(line)
        elif len(lines)>6:
            metadata = [lines[0]]
            metadata.append(lines[1]+' '+lines[2]+' '+lines[3].replace('\n',' '))
            for line in lines[4:]:
                if line != '' and line != '\n':
                    metadata.append(line)
        else:
            metadata=[line for line in lines if line != '' and line != '\n']
        print(metadata)
        cs.execute(
            "INSERT INTO meta_cat (domain, year, author, journal, source, article_name) VALUES (%s, %s, %s, %s, %s, %s)",
            ('Социология', 2017, metadata[2], 'Вестник Санкт-Петербургского университета. Социология.', metadata[0], metadata[1]))
    get_last_metas = "SELECT MAX(id_text) FROM meta_cat"
    cs.execute(get_last_metas)
    idtext = cs.fetchall()[0]['MAX(id_text)']
    conll = meta.replace('_meta','.pdf').replace("y_20",'y_')
    print(conll)
    with open(os.path.join(conll_dir, conll),'r',encoding='utf-8') as text:
        lines = text.read().splitlines()
        abs_sent_id = 1  # к какому по счету предложению в тексте относится слово
        for line in lines:
            if line != '' and line != '\n':
                if line[0] != '#':  # в формате udpipe номер предложения и текст закоменчены #
                    spl_line = line.split('\t')
                    try:
                        cs.execute(
                            "INSERT INTO words_cat (id_text, abs_sent_id, id_sent, word, lemma, POS_tag, morph, root_dist, sint_role) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                            (idtext, abs_sent_id, spl_line[0], spl_line[1], spl_line[2], spl_line[3], spl_line[5],
                             spl_line[6], spl_line[7]))
                    except:
                        print(spl_line)
            else:
                abs_sent_id += 1
                
connection.commit()

In [6]:
conll_dir = 'C:\\Users\\E6230\\udpiped\\psy'
metadir = 'C:\\Users\\E6230\\old_meta\\metas/metapsy'
connection = pymysql.connect(host='localhost',
                                 user='cat',
                                 password='12345678',
                                 db='cat',
                                 charset='utf8',
                                 cursorclass=pymysql.cursors.DictCursor)
cs = connection.cursor()
passed = set()
for meta in os.listdir(metadir):   
    conll = os.path.join(conll_dir, "cleaned_psy"+ meta.split('_')[1])
    if os.path.exists(conll):
        print(meta)
        with open(os.path.join(metadir,meta),'r',encoding='cp1251') as text:
            metadata = text.read().replace('\ufeff','').splitlines()
            
            article_name = metadata[1]
            if article_name in passed:
                continue
            print(metadata)
            passed.add(article_name)
            cs.execute(
                "INSERT INTO meta_cat (domain, year, author, journal, source, article_name) VALUES (%s, %s, %s, %s, %s, %s)",
                (metadata[4], int(metadata[3]), metadata[2].strip().upper(), metadata[5], metadata[0], article_name))
        get_last_metas = "SELECT MAX(id_text) FROM meta_cat"
        cs.execute(get_last_metas)
        idtext = cs.fetchall()[0]['MAX(id_text)']

        with open(conll,'r',encoding='utf-8') as text:
            lines = text.read().splitlines()
            abs_sent_id = 1  # к какому по счету предложению в тексте относится слово
            for line in lines:
                if line != '' and line != '\n':
                    if line[0] != '#':  # в формате udpipe номер предложения и текст закоменчены #
                        spl_line = line.split('\t')
                        try:
                            cs.execute(
                                "INSERT INTO words_cat (id_text, abs_sent_id, id_sent, word, lemma, POS_tag, morph, root_dist, sint_role) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                                (idtext, abs_sent_id, spl_line[0], spl_line[1], spl_line[2], spl_line[3], spl_line[5],
                                 spl_line[6], spl_line[7]))
                        except:
                            print(spl_line)
                else:
                    abs_sent_id += 1
                
connection.commit()

0_0.txt
['https://psy-journal.hse.ru/', 'ФЕНОМЕН «УХОДА В СВОИ МЫСЛИ» И ЕГО МЕСТО В КОНТИНУУМЕ СОЗНАНИЯ', 'Лапина Анна Андреевна, Чернышев Борис Владимирович', '2015', 'Психология и педагогика', 'Психология. Журнал Высшей школы экономики']
0_1.txt
0_2.txt
['http://www.psy.msu.ru/science/vestnik/index.html', 'Причины непосещаемости аудиторных занятий студентами', 'И.В. Харламенко, А.А. Гладышева', '2017', 'Психология и педагогика', 'Вестник Московского университета']
0_62.txt
['http://www.psy.msu.ru/science/vestnik/index.html', 'Признаки Парадигмы и обоснование третьей Парадигмы в Психологии', 'в. м. еськов, Ю. П. зинченко, о. е. Филатова', '2017', 'Психология и педагогика', 'Вестник Московского университета']
0_63.txt
['http://www.psy.msu.ru/science/vestnik/index.html', 'Персонологическая рефлексия жизнетворчества П.Я. Гальперина и его Психолого-Педагогической и историко-методологической деятельности в московском университете', 'и. н. семенов', '2017', 'Психология и педагогика', 'Вестн